In [1]:
from setup import init_django

In [2]:
init_django()

In [3]:
from market.models import StockQuote

In [4]:
from django.db.models import Avg, F, RowRange, Window, Max
from django.db.models.functions import TruncDate
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [5]:
days_ago = 100
now = timezone.now()
start_date = now - timedelta(days=days_ago)
end_date = now

qs = StockQuote.objects.filter(time__range=(start_date, end_date))
qs.count()

357600

In [6]:
total = 0
for obj in qs:
    total += obj.close_price


In [7]:
total

Decimal('15421378769.2939')

In [8]:
total / qs.count()

Decimal('43124.66098795833333333333333')

In [9]:
qs.values('company').annotate(avg_price=Avg('close_price'))

<QuerySet [{'company': 1, 'avg_price': Decimal('108172.568602562567')}, {'company': 2, 'avg_price': Decimal('0.14481598640891440149')}, {'company': 3, 'avg_price': Decimal('460.8797860469108779')}, {'company': 4, 'avg_price': Decimal('0.25478290454098130184')}]>

In [10]:
count = 5
ticker = "X:BTCUSD"
rolling_qs = list(qs.filter(company__ticker = ticker).order_by('-time')[:count])
rolling_qs.reverse()
rolling_qs

[<StockQuote: StockQuote object (3407989)>,
 <StockQuote: StockQuote object (3407990)>,
 <StockQuote: StockQuote object (3407991)>,
 <StockQuote: StockQuote object (3407992)>,
 <StockQuote: StockQuote object (3407993)>]

In [11]:
total = 0
for i, obj in enumerate(rolling_qs):
    total += obj.close_price
    avg = Decimal(total / (i + 1))
    print(i+1, obj.id, obj.close_price, avg)

1 3407989 114116.9000 114116.9000
2 3407990 114141.4000 114129.1500
3 3407991 114134.3800 114130.8933333333333333333333
4 3407992 114141.0200 114133.4250
5 3407993 114180.0000 114142.7400


In [12]:
latest_daily_timestamps = (
    StockQuote.objects.filter(company__ticker = ticker, time__range=(start_date, end_date))
    .annotate(date=TruncDate('time'))
    .values('company', 'date')
    .annotate(latest_time=Max('time'))
    .values('company', 'date', 'latest_time')
    .order_by('date')
)
actual_timestamps = [x['latest_time'] for x in latest_daily_timestamps]
actual_timestamps

[datetime.datetime(2025, 4, 29, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 4, 30, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 5, 1, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 5, 2, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 5, 3, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 5, 4, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 5, 5, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 5, 6, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 5, 7, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 5, 8, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 5, 9, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 5, 10, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 5, 11, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 5, 12, 23, 59, tzinfo=datetime.timezone.utc),
 datetime.datet

In [13]:
qs = StockQuote.objects.filter(
    company__ticker=ticker, 
    time__range=(start_date, end_date),
    time__in=actual_timestamps
    )

In [14]:
frame_start = -(count - 1)
ma_val = qs.annotate(
    ma=Window(
        expression=Avg('close_price'),
        order_by=F('time').asc(),
        partition_by=[],
        frame=RowRange(start=frame_start, end=0),
    )
).order_by('-time')

In [15]:
for i, obj in enumerate(ma_val[:10]):
    print(i, obj.id, obj.close_price, obj.ma, obj.time)


0 3407993 114180.0000 113844.302000000000 2025-08-05 23:59:00+00:00
1 3406553 115051.8500 114160.528000000000 2025-08-04 23:59:00+00:00
2 3405113 114215.7300 114716.188000000000 2025-08-03 23:59:00+00:00
3 3403673 112525.2000 115459.720000000000 2025-08-02 23:59:00+00:00
4 3402233 113248.7300 116550.620000000000 2025-08-01 23:59:00+00:00
5 3400793 115761.1300 117793.978000000000 2025-07-31 23:59:00+00:00
6 3399353 117830.1500 118229.252000000000 2025-07-30 23:59:00+00:00
7 3397913 117933.3900 118189.876000000000 2025-07-29 23:59:00+00:00
8 3396473 117979.7000 118282.606000000000 2025-07-28 23:59:00+00:00
9 3395033 119465.5200 118448.754000000000 2025-07-27 23:59:00+00:00


In [16]:
ma_vals = qs.annotate(
    ma_5=Window(
        expression=Avg('close_price'),
        order_by=F('time').asc(),
        partition_by=[],
        frame=RowRange(start=-(5-1), end=0),
    ),
    ma_20=Window(
        expression=Avg('close_price'),
        order_by=F('time').asc(),
        partition_by=[],
        frame=RowRange(start=-(20-1), end=0),
    )
).order_by('-time').first()

In [17]:
print(ma_vals.id, ma_vals.close_price, ma_vals.ma_5, ma_vals.ma_20)

3407993 114180.0000 113844.302000000000 117046.271000000000
